# Ladda ner, läs in data, tvätta och spara ner
Detta skript laddar ner rätt filer. Läser in och tvättar dem. Sparar i lämpligt format. Med lite tur behöver man bara köra skriptet en gång ;)

In [10]:
import pandas as pd
from pathlib import Path
import re
import pandas as pd
import tqdm
import requests, zipfile, io

#PATH_TO_FILES=Path('C:\Datalab\sida_projekt\sida_projekt\Riksdagsdata\Propositioner') # Byt ut denna mot PATH_TO_PROP för alla utom Joakim
PATH_TO_RAW=Path('raw')
PATH_TO_OUTDATA=Path('data')
PATH_TO_PROP=Path('raw') / 'propositioner'
PATH_TO_META=Path('meta')

# Skippa dokument med titlar som innehåller:
FILTER_TITLE='vårproposition|budgetproposition|ändringsbudget' # Lämna tom sträng om du inte vill filtrera


## Definiera funktioner som vi kommer använda

In [13]:

def downoad_download_meta(PATH_TO_FILES=PATH_TO_META):

    # Compile lists
    prop_list=[]
    for x in range(1998,2011,4):
        prop_list.append(f"https://data.riksdagen.se/dataset/dokument/prop-{x}-{x+3}.csv.zip")

    sou_list=[]
    for x in ['2020-','2015-','2010-2014','2005-2009','2000-2004']:
        sou_list.append(f'https://data.riksdagen.se/dataset/dokument/sou-{x}.csv.zip')


    # fetch data
    for url in prop_list+sou_list:  
        filename=Path(url).stem  
        print('Downloading',filename)
        df=pd.read_csv(url,header=None)
        df.to_csv(PATH_TO_FILES / filename)
        print('Done!')


def read_meta_to_df(PATH_TO_META=PATH_TO_META,FILTER_TITLE=''):

    """Concatenates all csv-files in PATH_TO_META into a common df. FILTER_TITLE is optional.
    returns: df
    """
        
    df=pd.concat([pd.read_csv(p, usecols=range(1,18)) for p in PATH_TO_META.glob('*.csv')])
    colnames=['hangar_id', 'dok_id', 'rm', 'beteckning', 'doktyp', 'typ', 'subtyp', 'tempbeteckning', 'organ', 'mottagare', 'nummer', 'datum', 'systemdatum', 'titel', 'subtitel', 'status', 'relaterat_id']
    df.columns=colnames
    df.set_index('hangar_id', inplace=True)
    print('Raw input shape', df.shape)

    # Filter out
    if FILTER_TITLE!='':    
        df=df.loc[~df['titel'].str.contains(FILTER_TITLE,flags=re.I),:]
        print('Output shape:',df.shape)
    else:
        print('No filters where used.')

    print('Column names:', colnames)
    return df

def downoad_starter_pack(PATH_TO_FILES=PATH_TO_PROP):

    # Metadata
    csv_url=["https://data.riksdagen.se/dataset/dokument/prop-2018-2021.csv.zip","https://data.riksdagen.se/dataset/dokument/prop-2014-2017.csv.zip"]
    for url in csv_url:    
        df=pd.read_csv(url,header=None)
        df.to_csv(PATH_TO_FILES / Path(url).stem)

    # text documents
    text_url=["https://data.riksdagen.se/dataset/dokument/prop-2018-2021.text.zip","https://data.riksdagen.se/dataset/dokument/prop-2014-2017.text.zip"]
    for url in text_url:    
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(PATH_TO_FILES)

def assert_folders(PATH_TO_RAW=PATH_TO_RAW,PATH_TO_PROP=PATH_TO_PROP):
    Path(PATH_TO_RAW).mkdir(exist_ok=True)
    Path(PATH_TO_PROP).mkdir(parents=True,exist_ok=True)

def read_to_df(PATH_TO_FILES=PATH_TO_PROP,FILTER_TITLE=''):
    
    # Read metadata
    df1=pd.read_csv(PATH_TO_FILES / 'prop-2018-2021.csv',header=None)
    df2=pd.read_csv(PATH_TO_FILES /'prop-2014-2017.csv',header=None)
    df=pd.concat([df1,df2])
    print('Raw input shape', df.shape)

    #Pre-processing
    df.loc[:,1]=df.loc[:,1].str.lower()
    df.set_index(1, inplace=True)
    df.index.rename('filename', inplace=True)
    df=df.rename(columns={2:'rm',8:'Departement',9:'Utskott',13:'Titel'})

    # Filter out
    df=df.loc[~df['Titel'].str.contains(FILTER_TITLE,flags=re.I),['Titel','rm','Departement','Utskott']]
    print('Output shape:',df.shape)
    return df

def add_space(text):
    """ Adds space before (suspected) chapter"""
    return re.sub('\.\s*(\n[A-Ö\d][\.]?\w*:?( [\w\-–:]*){0,7}\s*?\n+)[A-Ö]',r'\n\1',text)

def remove_chapnumber(text):
    """Remove chapter number on single row
    Returns cleaned text"""
    return re.sub('\n\d+\.\d+\n|\n\d+\n','',text)

def avstava(text):
    
    # Ta bort sidnummer om det kommer in i en avstavning
    text=re.sub('([a-ö]{2,}-\s*)(\d+)',r'\1',text)

    streck=re.compile(r"""  (?!it)          # Matcha inte på it
                            (               # Grupp 1 innehåller grupp 2 till 4:
                                ([a-ö]{2,})     # Grupp 2 första delen av det eventuellt avstavade ordet (kräver minst 2 bokstäver)
                                (-\s*\d*\s*)    # Grupp 3 vill vi ta bort från hela grupp 0 (fångar även insprängda sidnummer)
                                ([a-ö]*)        # Grupp 4 Sista delen i det eventuellt avstavade ordet
                                )"""            # Notera att vi, i grupp 4, endast matchar på små bokstäver från a till ö
        ,                                       
        re.X + re.U
    )       
        
    def dashrepl(matchobj):
        """ Fixar avstavningar """
        if matchobj.group(4) is None:
            return matchobj.group(0)
        elif matchobj.group(2).islower():
            if (matchobj.group(2) in ['bnp','tfp']) or (matchobj.group(4) in ['och','eller']): # Manuella tillägg av godkända prefix och ord efter bindesstrecket som är tillåtna.
                return matchobj.group(0)
            else:
                return matchobj.group(2) + matchobj.group(4)
        else:
            return matchobj.group(0) 

    return re.sub(streck,dashrepl,text)

def cleaner(text):
    """Chained cleaner functions"""
    return add_space(remove_chapnumber(avstava(text)))

def import_and_clean_txt(df, PATH_TO_FILES=PATH_TO_PROP,clean=True):
    for ids, row in tqdm(df.iterrows(), total=df.shape[0]):
        try:    
            with open(PATH_TO_FILES / (ids + '.txt'), encoding='utf8') as file:
                if clean:
                    df.at[ids,'text']=cleaner(file.read())
                else:
                    df.at[ids,'text']=file.read()
        except FileNotFoundError as ex:
            print(ex)
            print(ids, row['Titel'])
            df.at[ids,'text']='' # File not found
    print('Tomma dokument')
    print(df.loc[df.text==''])
    print('Tar bort tomma dokument...')

    return df.loc[~(df.text=='')]

## 1. Kör detta om det är första gången du starta projektet
__assert_folders()__ sätter upp rät mappstruktur

__download_starter_pack()__ laddar ner  zipfiler med csv och text-filer i rätt mappar

In [73]:
assert_folders() # Creates folders if needed
downoad_starter_pack(PATH_TO_FILES=PATH_TO_PROP)
downoad_download_meta()

## 2. Även denna cell behöver du bara köra en gång
__read_to_df()__ läser in metadata från csv-filerna

__import_and_clean()__ lägger till alla textdokument till datatabellen *df* och gör vissa rensningar av texterna

Sista cellen sparar filen som en pickle.

In [ ]:
df=read_to_df(PATH_TO_FILES=PATH_TO_PROP,FILTER_TITLE=FILTER_TITLE)
df=import_and_clean_txt(df)
print('df.shape:',df.shape)

In [30]:

df.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,2222687,GM031,1998/99,1,prop,prop,prop,NaN,Finansdepartementet,NaN,1,1999-01-01 00:00:00,2006-10-23 14:13:40,Budgetpropositionen för 1999,"Förslag till statsbudget, finansplan m.m. (1. ...",NaN,NaN
1,2291126,GM0310,1998/99,10,prop,prop,prop,NaN,Justitiedepartementet,NaN,10,1999-01-01 00:00:00,2006-10-23 14:13:40,Ändringar i rättshjälpslagen,NaN,NaN,NaN
2,2294100,GM03100,1998/99,100,prop,prop,prop,NaN,Finansdepartementet,NaN,100,1999-01-01 00:00:00,2006-10-23 14:13:40,1999 års ekonomiska vårproposition,NaN,NaN,NaN
3,2457451,GM03100D1,1998/99,100D1,prop,prop,prop,100.1,Finansdepartementet,NaN,100,1999-01-01 00:00:00,2006-10-23 14:13:40,1999 års ekonomiska vårproposition,Svensk ekonomi,NaN,NaN
4,2457452,GM03100D2,1998/99,100D2,prop,prop,prop,100.2,Finansdepartementet,NaN,100,1999-01-01 00:00:00,2006-10-23 14:13:40,1999 års ekonomiska vårproposition,Avstämning av målet om en halverad öppen arbet...,NaN,NaN


In [45]:
df['subtyp'].value_counts(dropna=False)

sou     3257
prop    3019
skr      624
NaN       40
Name: subtyp, dtype: int64

sou     3257
prop    3019
skr      624
NaN       40
Name: subtyp, dtype: int64

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,NaN,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
1,0.0,2222687,GM031,1998/99,1,prop,prop,prop,NaN,Finansdepartementet,NaN,1,1999-01-01 00:00:00,2006-10-23 14:13:40,Budgetpropositionen för 1999,"Förslag till statsbudget, finansplan m.m. (1. ...",NaN,NaN
2,1.0,2291126,GM0310,1998/99,10,prop,prop,prop,NaN,Justitiedepartementet,NaN,10,1999-01-01 00:00:00,2006-10-23 14:13:40,Ändringar i rättshjälpslagen,NaN,NaN,NaN
3,2.0,2294100,GM03100,1998/99,100,prop,prop,prop,NaN,Finansdepartementet,NaN,100,1999-01-01 00:00:00,2006-10-23 14:13:40,1999 års ekonomiska vårproposition,NaN,NaN,NaN
4,3.0,2457451,GM03100D1,1998/99,100D1,prop,prop,prop,100.1,Finansdepartementet,NaN,100,1999-01-01 00:00:00,2006-10-23 14:13:40,1999 års ekonomiska vårproposition,Svensk ekonomi,NaN,NaN


In [15]:
df_meta=read_to_df(PATH_TO_FILES=PATH_TO_META,FILTER_TITLE='')
df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'meta\\prop-2018-2021.csv'

In [41]:
# Pickle vs. parquet save: 2.5s vs. 4.8s
df.to_pickle(PATH_TO_OUTDATA / 'data.pkl')

# Pickle vs. parquet load: 1.8 vs. 3.6
# df=pd.read_pickle(PATH_TO_OUTDATA / 'data.pkl')